In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [107]:
data = pd.read_csv('biddings.csv')
dfo = pd.DataFrame(data)


In [108]:
df = dfo.iloc[:250000, ] 
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
249995,-0.02,2.39,-1.20,-0.69,0.94,0.57,0.19,0.03,-0.41,-0.53,...,0.08,-0.07,-0.31,-1.00,0.09,-0.76,-0.10,0.05,-0.05,0
249996,-0.01,2.03,-1.99,-0.63,0.77,-1.61,0.10,1.64,-0.28,2.22,...,-0.03,-0.02,0.04,0.12,-0.05,0.07,0.06,0.66,-0.18,0
249997,-0.02,2.04,2.11,-0.30,0.14,-0.59,0.31,-0.80,0.11,0.23,...,0.09,-1.03,-0.55,0.05,0.39,0.01,-0.68,0.26,-0.13,0
249998,-0.01,-2.39,2.04,-0.09,-0.22,-0.58,0.16,-1.47,0.37,-0.67,...,0.06,0.92,-0.62,-0.14,-1.04,2.37,-0.44,-0.57,0.12,0
249999,-0.01,0.80,-4.22,-0.21,-0.69,2.71,2.85,-0.51,-0.33,0.08,...,-0.15,0.17,0.67,0.15,2.70,0.23,0.12,1.49,-0.46,0


In [75]:
#Convert column = shown ad was clicked: 1 = clicked ad
df.convert.value_counts()

0    998092
1      1908
Name: convert, dtype: int64

### Severely underbalanced

Converted accounts for 1908/999999 = .191% of our data

We will have to oversample or undersample our target. Most likely undersample to minimize our noise

I will use RandomUnderSampler, NearMiss, and EditedNearestNeighbors
to underersample our target here first to get baselines

In [103]:
from sklearn.model_selection import train_test_split

# x = df.drop(['convert'],axis=1) #training features
# y = df.convert #target

#Need to set our train/test values before undersampling
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled,
                                                  test_size = .10)

In [105]:
print(sorted(Counter(y_test).items()))


[(0, 196), (1, 186)]


In [96]:
y_test.value_counts()

0    1248
1       2
Name: convert, dtype: int64

In [97]:
from collections import Counter
from imblearn.under_sampling import NearMiss

nm1 = NearMiss(version=3)
X_resampled, y_resampled = nm1.fit_resample(x, y)
print(sorted(Counter(y_resampled).items()))



[(0, 1908), (1, 1908)]


In [106]:
#First model using our undersampled data (RandomUnderSampler)
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC()
clf.fit(x_train, y_train) 

print('Training accuracy LinearSVC is: {}'.format(clf.score(x_train, y_train)))
print('Test accuracy LinearSVC is: {}'.format(clf.score(x_test, y_test)))

scores = cross_val_score(clf, X_resampled, y_resampled, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy LinearSVC is: 0.675596971461852
Test accuracy LinearSVC is: 0.6596858638743456
Cross Val LinearSVC Accuracy: 0.63 (+/- 0.17)
              precision    recall  f1-score   support

           0       0.67      0.66      0.67       196
           1       0.65      0.66      0.65       186

   micro avg       0.66      0.66      0.66       382
   macro avg       0.66      0.66      0.66       382
weighted avg       0.66      0.66      0.66       382



In [79]:
from sklearn import ensemble

params = {'n_estimators': 750,
          'max_features': 'log2',
          'max_depth': 6}

rfc = ensemble.RandomForestClassifier(**params)
rfc.fit(X_resampled, y_resampled)

print('Training accuracy normal RandomForest is: {}'.format(rfc.score(X_resampled, y_resampled)))
print('Test accuracy normal RandomForest is: {}'.format(rfc.score(x_test, y_test)))

scores = cross_val_score(rfc, X_resampled, y_resampled, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

y_pred = rfc.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy normal RandomForest is: 0.7656158785755983
Test accuracy normal RandomForest is: 0.35704
Cross Val LinearSVC Accuracy: 0.65 (+/- 0.17)
              precision    recall  f1-score   support

           0       1.00      0.36      0.53     99805
           1       0.00      0.63      0.00       195

   micro avg       0.36      0.36      0.36    100000
   macro avg       0.50      0.49      0.26    100000
weighted avg       1.00      0.36      0.52    100000



In [80]:
params = {'n_estimators': 750,
          'max_features': 'log2',
          'max_depth': 6}

clf1 = ensemble.GradientBoostingClassifier(**params)
clf1.fit(X_resampled, y_resampled)

print('Training accuracy normal GradientBoosting is: {}'.format(clf1.score(X_resampled, y_resampled)))
print('Test accuracy normal GradientBoosting is: {}'.format(clf1.score(x_test, y_test)))

y_pred = clf1.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy normal GradientBoosting is: 0.9985405720957384
Test accuracy normal GradientBoosting is: 0.30918
              precision    recall  f1-score   support

           0       1.00      0.31      0.47     99805
           1       0.00      0.68      0.00       195

   micro avg       0.31      0.31      0.31    100000
   macro avg       0.50      0.49      0.24    100000
weighted avg       1.00      0.31      0.47    100000



In [22]:
scores = cross_val_score(clf1, X_resampled, y_resampled, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Cross Val LinearSVC Accuracy: 0.94 (+/- 0.09)


In [33]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [750],
    'max_features': ['log2'],
    'max_depth' : [4,5,6]
}

CV_clf = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 6)
CV_clf.fit(X_resampled, y_resampled)

CV_clf.best_params_

{'max_depth': 6, 'max_features': 'log2', 'n_estimators': 750}

In [85]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

y_pred = clf.predict(x_test) #predict y based on x_test
balanced_accuracy_score(y_test, y_pred)

0.5047423244557656

In [86]:
f1_score(y_test, y_pred)

0.00393927619149706

In [87]:
%timeit clf.predict(df.iloc[5:6, 0:88] )

1.65 ms ± 423 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


366